In [ ]:
import fimdp
from fimdp.core import ConsMDP
#fixpoints = fimdp.fixpoints
#fimdp.dot.dotpr = 'neato'
from fimdp.energy_solvers import BasicES
from fimdp.objectives import *

In [ ]:
def consMDP_flower(cap=8):
    m = ConsMDP()
    m.dot_pr = "neato"
    m.new_states(2)
    m.add_action(1,{0:1},"a",1)
    m.add_action(0,{1:1},"t",cap)
    m.set_reload(1)

    for c in range(1,cap):
        s = m.new_state(reload=True)
        m.add_action(0,{s:1},f"{s}",cap-c)
        m.add_action(s,{0:1},"a",c-1)
    return m

In [ ]:
consMDP_flower()

## Double flower shaped consMDP
Computing positive reachability may involve up to quadratic number of fixpoint-iterations (and thus cubic complexity in general). To be more precise, for $R$ being the set of reloads and $Q$ being the set of non-reload staes, the number of iterations $i$ is:
$$
i = |R| \cdot \frac{|P|}{2}
$$

In [ ]:
def consMDP_double_flower(cap=32,path=3):
    m = ConsMDP()
    m.dot_pr = "neato"
    m.new_states(2)
    #m.add_action(1,{0:1},"a",1)
    #m.add_action(0,{1:1},"t",cap)

    for c in range(2,cap, 2):
        s = m.new_state(reload=True, name = f"{c}")
        h = (c//2) % 2
        m.add_action(h,{s:1},f"{s}",cap-c)
        m.add_action(s,{h:1},"a",c-1)
        
    prev_o = 1
    prev_e = 0
    for p in range(path):
        curr_o = m.new_state()
        curr_e = m.new_state()

        m.add_action(prev_o,{curr_o:1},"p",0)
        m.add_action(prev_e,{curr_e:1},"p",0)
        
        prev_o = curr_o
        prev_e = curr_e
        
    m.add_action(prev_o,{0:1},"p",1)
    m.add_action(prev_e,{1:1},"p",1)
    
    return m

In [ ]:
cap = 32 # We have cap/2 reload states, cap/4 in each flower
path = 6
m = consMDP_double_flower(cap, path)
solver = BasicES(m, cap=cap, targets=[2])
# This will yield ∞ for most states, we need at least cap+2
print(solver.get_min_levels(POS_REACH))
solver

In [ ]:
# Let's see the number of iterations needed
solver = BasicES(mdp=m, cap=cap+2, targets=[2])
solver.debug = True
print(solver.get_min_levels(POS_REACH))
solver.show()

### Visual evolution of the computation
Let's have some smaller double-flower and see the results step-by-step

In [ ]:
cap = 16 # We have cap/2 reload states, cap/4 in each flower
path = 3
m = consMDP_double_flower(cap, path)

solver = BasicES(m, cap+2, set([2]))
solver.get_min_levels(MIN_INIT_CONS)

# Turn on the debug visualizations
solver.debug_vis=True
solver.get_min_levels(SAFE)

In [ ]:
solver.get_min_levels(POS_REACH)
solver

In [ ]:
solver.get_min_levels(AS_REACH)